# Descarga de Datos
A la hora del analísis de los datos, después una vez seleccionado el problema es la extracción de datos. 

¿Dónde podemos encontrar información para resolver nuestro problema?. En nuestro caso partiremos de la base de datos del SIAP (Servicio de Informacion Agroalimentaria y Pesquera de la Secretaria de Agricultura y Desarrollo Rural).

Otra pregunta es, ¿Qué información nos será de utilidad?, por lo general no se requiere la base de datos completa, entonces viene bien descargar la información relacionada o útil para la resolución. 

En esta libreta, pretendemos construir una forma fácil de seleccionar parte de la base de datos para su futura utilización.

## Importación de Librerias
Para la descarga usaremos las siguientes líbrerias.

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from lxml import etree
from bs4 import BeautifulSoup
import os
import json
import time
import datetime

"""
El siguiente fragmento de código pretende limpiar la consola de los problemas de certificados
que tiene la página origen de los datos del SIAP. 
"""

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


Una vez hayan sido seleccionadas las librerias requeridas, iremos describiendo el paso de construcción paso a paso.
## Carpeta de Destino.

Lo primero, es definir la carpeta donde serán descargados los archivos. Nosotros la llamaremos {\em selection}, dentro de la caperta siap, que a su vez esté dentro de la carpeta data.

In [2]:

data_dir = "./data/siap/selection"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Constucción del selector

Una de la forma más sencilla de generar un selector es mediante un diccionario, donde cada clave es una de las categorías que nos interesan y su valor será una lista donde especificaremos la información requerida. 

Entonces, de la pagina del SIAP, extraemos los distintos parámetros que se pueden modificar. Estos parámetros toman el nombre de 'ids' dentro de la base de datos del SIAP, entonces vamos a tomarlos de los ids extraidos y almacenados en 'idsSIAP.json'.

In [5]:
with open("idsSIAP.json",encoding= 'utf8') as json_file:
    dict_ids = json.load(json_file)

dict_cultivos = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("cultivo")}
dict_ciclos = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("ciclo")}
dict_mes = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("mes")}
dict_modalidad = {k.split('_')[1]:v for k,v in dict_ids.items() if k.startswith("modalidad")}

url = "https://nube.siap.gob.mx/avance_agricola/"

payload = {'xajax': 'reporte', # Para obtener la tabla
'xajaxr': '1696449941927', # Timestamp UNIX
'xajaxargs[]': [
    '1', # 1: Desglose por estados, 2: Cultivo total
    '2023', # Anio
    '5', # ID Ciclo
    '3', # ID Modalidad
    '0', # ID Estado (0: Nacional)
    '--',
    '--',
    '7', # ID Cultivo
    '200201',
    '0',
    '1', # 1: Por municipio
    'undefined',
    'undefined',
    'undefined',
    '8' # Valor del mes
    ]
}

headers = {
  'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

Entonces vemos 5 de las categorías principales.
* Cultivos
* Ciclos
* Mes
* Modalidad

Además de elementos como 'url', 'payload' y 'headers'. Ahora, nuestro intéres esta en 'payload' ya que este es quien da formato a las peticiones de la base de datos. Notamos que agrega el año como categoría, luego elementos adicionales como el desglose, en este caso puede ser mediante estados o directamente el nacional. Por el momento nos centraremos en las primeras 5 categorías. 

* Año
* Cultivos
* Ciclos
* Mes
* Modalidad

Ahora mostraremos el formato general de nuestro selector.

In [ ]:
selector_data = {
    'Anio':[],
    'Ciclos':['Otoño - Invierno'],
    'Cultivos':['Aguacate'],
    'Meses':['Enero'],
    'Modalidad':['Riego']
}

Notemos que el selector, posee parámetros con listas vacías. Bien, en nuestro selector, queremos que esa lista vacía, de la orden que considere todas las opciones posibles, en este caso, los años, para eso consideraremos un selector total, es decir, uno que considere la base de datos completa.

In [6]:
ciclos = list(dict_ciclos.keys())
cultivos = list(dict_cultivos.keys())
meses = list(dict_mes.keys())
modalidad = list(dict_modalidad.keys())
anios = [2018 + i for i in range(6)]

all_data = {
    'Anio':anios,
    'Ciclos':ciclos,
    'Cultivos':cultivos,
    'Meses':meses,
    'Modalidad':modalidad
}

Con esto, y algunas funciones defenidas en `selector.py`, tenemos la función que buscamos. 

In [8]:
import selector as s

def download_select_data(selector):
    sel_keys = selector.keys()
    for u in sel_keys:
        if len(selector[u]) == 0:
            selector[u] = all_data[u]
    for year in selector['Anio']:
        payload['xajaxargs[]'][1] = str(year)
        for key_mes in selector['Meses']:
            payload['xajaxargs[]'][14] = dict_mes[key_mes]

            for key_cultivo in selector['Cultivos']:
                if dict_cultivos[key_cultivo] == "0":
                    continue
                    
                if os.path.exists(f"{data_dir}/{key_cultivo.lower().replace(' ','_')}_{key_mes.lower()}_{year}.csv"):
                    continue

                payload['xajaxargs[]'][7] = dict_cultivos[key_cultivo]

                response = requests.request("POST", url, headers=headers, data=payload, verify=False)
                response.encoding='ISO-8859-1'

                #extract the cdata as a string
                root = etree.fromstring(response.content)
                cd = root.xpath('//cmd//text()')[0]

                parsed_html = BeautifulSoup(str(cd),"lxml")
                parsed_table = parsed_html.body.find("table")

                if parsed_table is None:
                    continue

                df_iter = pd.read_html(str(parsed_html.body.find("table")))[0]

                df_iter.to_csv(f"{data_dir}/{key_cultivo.lower().replace(' ','_')}_{key_mes.lower()}_{year}.csv",index=False)
                time.sleep(0.1)


Ahora, recordando el selector de ejemplo, notemos que se descargarán únicamente la información requerida.

In [9]:
selector_data = {
    'Anio':[],
    'Ciclos':['Otoño - Invierno'],
    'Cultivos':['Aguacate'],
    'Meses':['Enero'],
    'Modalidad':['Riego']
}

download_select_data(selector_data)

El selector puede agregar más o menos categorías según lo requerido. Ahora, en la siguiente libreta, veremos la relevancia del selector. 